# PubMed Open Access Alzheimer's Disease Articles: Query and NLP Augmented Parsing Example

This notebook demonstrates how to use the `PubMedQueryService`, `PubMedAnalyzer`, and `PMCFullTextParser` classes to:
- Search PubMed for open access articles related to Alzheimer's disease (MeSH term) from the past 10 years
- Fetch and summarize article metadata
- Filter articles by keywords
- Retrieve full text from PMC
- Parse and extract tables, sections, and figures from full text XML
- Analyze and visualize article metadata distributions

## 1. Initialize environment

* Pull NCBI API Key from .env file

In [1]:

import os
import logging
import sys
from dotenv import load_dotenv

DEBUG = True

# configure logging
logging.basicConfig(
    level=logging.DEBUG if DEBUG else logging.INFO,
    stream=sys.stdout,
    format="%(asctime)s %(levelname)s %(name)s: %(message)s"
)

# set your email and API key for NCBI from environment variables
EMAIL = os.environ.get("EMAIL")  # e.g., 'your.email@example.com'
API_KEY = os.environ.get("NCBI_API_KEY")  # e.g., 'your_ncbi_api_key'


## 2. Initialize PubMedQueryService

Create an instance of PubMedQueryService. Optionally, provide your email and NCBI API key for higher rate limits.

In [2]:
from niagads.pubmed.services import PubMedQueryService

pubmed_service = PubMedQueryService(email=EMAIL, api_key=API_KEY, debug=DEBUG)

## 3. Search PubMed for Articles (Current Year Only)

Search for open access PubMed articles with the MeSH term 'Alzheimer Disease' for the current year only. This step fetches all PMIDs for the current year.

In [3]:
from niagads.pubmed.services import PubMedQueryFilters

# Search for open access Alzheimer's disease genetics articles (MeSH) for the current year only
mesh_term = ["Alzheimer Disease", "Genetics"]
current_year = 2025

# Get article count for the current year
count = await pubmed_service.find_articles(
    filters=PubMedQueryFilters(
        keyword=None,
        year=current_year,
        mesh_term=mesh_term,
        open_access_only=True
    ),
    counts_only=True
)
print(f"{current_year}: {count} articles found")

# Fetch all PMIDs for the current year (if any)
if count > 0:
    all_pmids = await pubmed_service.find_articles(
        filters=PubMedQueryFilters(
            keyword=None,
            year=current_year,
            mesh_term=mesh_term,
            open_access_only=True
        ),
        max_results=count,
        ids_only=True
    )
    print(f"Total PMIDs found: {len(all_pmids)} (should match count: {count})")
else:
    all_pmids = []
    print("No articles found for the current year.")

2025-11-17 17:11:41,071 DEBUG niagads.common.core: PubMedQueryService:_rate_limited_request Requesting: https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi with params: {'db': 'pubmed', 'term': '2025[pdat] AND Alzheimer Disease[MeSH Terms] AND Genetics[MeSH Terms] AND free full text[filter]', 'retmode': 'xml', 'rettype': 'count', 'retmax': 0}
2025-11-17 17:11:41,221 DEBUG niagads.common.core: PubMedQueryService:_rate_limited_request Response received for /entrez/eutils/esearch.fcgi: <?xml version="1.0" encoding="UTF-8" ?>
<!DOCTYPE eSearchResult PUBLIC "-//NLM//DTD esearch 20060628//EN" "https://eutils.ncbi.nlm.nih.gov/eutils/dtd/20060628/esearch.dtd">
<eSearchResult>
	<Count>146</Count>
</eSearchResult>
...
2025: 146 articles found
2025-11-17 17:11:41,562 DEBUG niagads.common.core: PubMedQueryService:_rate_limited_request Requesting: https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi with params: {'db': 'pubmed', 'term': '2025[pdat] AND Alzheimer Disease[MeSH Terms] AND

## 4. Fetch Article Metadata

Fetch metadata (title, abstract, authors, etc.) for the PMIDs found in the previous step.

In [4]:
# Fetch metadata for the first 20 PMIDs as a sample
sample_pmids = all_pmids[:20]

async def fetch_sample_metadata():
    metadata = await pubmed_service.fetch_article_metadata(sample_pmids)
    print(f"Fetched metadata for {len(metadata)} articles.")
    for article in metadata:
        print(f"PMID: {article.pmid}")
        print(f"Title: {article.title}")
        print(f"Year: {article.year}")
        print(f"Journal: {article.journal}")
        print(f"Authors: {[a.last for a in article.authors]}")
        print(f"Abstract: {article.abstract[:200] if article.abstract else ''}...\n")
    return metadata

sample_metadata = await fetch_sample_metadata()

2025-11-17 17:11:43,227 DEBUG niagads.common.core: PubMedQueryService:_rate_limited_request Requesting: https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi with params: {'db': 'pubmed', 'id': '41235755,41225772,41216795,41212591,41204210,41198777,41197609,41178698,41174000,41173857,41173821,41157910,41146472,41091855,41085772,41085205,41078215,41073674,41042700,41035143', 'retmode': 'xml'}
2025-11-17 17:11:43,521 DEBUG niagads.common.core: PubMedQueryService:_rate_limited_request Response received for /entrez/eutils/efetch.fcgi: <?xml version="1.0" ?>
<!DOCTYPE PubmedArticleSet PUBLIC "-//NLM//DTD PubMedArticle, 1st January 2025//EN" "https://dtd.nlm.nih.gov/ncbi/pubmed/out/pubmed_250101.dtd">
<PubmedArticleSet>
<PubmedArticle><MedlineCitation Status="MEDLINE" Owner="NLM" IndexingMethod="Automated"><PMID Version="1">41235755</PMID><DateCompleted><Year>2025</Year><Month>11</Month><Day>14</Day></DateCompleted><DateRevised><Year>2025</Year><Month>11</Month><Day>16</Day></DateRevised

## 5. Filter Articles by Keywords

Filter the fetched articles by keywords in the title or abstract (e.g., 'amyloid', 'tau').

In [5]:
# Filter articles by keywords in title or abstract
keywords = ["amyloid", "tau"]
filtered_articles = PubMedQueryService.filter_articles_by_keywords(sample_metadata, keywords)

print(f"Articles matching keywords {keywords}: {len(filtered_articles)}")
for article in filtered_articles:
    print(f"PMID: {article.pmid} | Title: {article.title}")

Articles matching keywords ['amyloid', 'tau']: 5
PMID: 41178698 | Title: Proteomic subtyping of Alzheimer's disease CSF links blood-brain barrier dysfunction to reduced levels of tau and synaptic biomarkers.
PMID: 41174000 | Title: Combined multi-omics and brain pathology reveal novel biomarkers for alzheimer's disease.
PMID: 41085772 | Title: Genetic and proteomic analysis identifies BAG3 as an amyloid-responsive regulator of neuronal proteostasis.
PMID: 41078215 | Title: Differential effects of age and sex on tau pathology propagation in the htau mouse model: A neuropathological and proteomic study.
PMID: 41073674 | Title: Glia inflammation and cell death pathways drive disease progression in preclinical and early AD.


## 6. Summarize and Plot Article Metadata

Use PubMedAnalyzer to summarize the distribution of publication years, journals, and MeSH terms, and plot the results.

In [6]:
# Summarize and plot article metadata using PubMedAnalyzer
from niagads.pubmed.parsers import PubMedAnalyzer
analyzer = PubMedAnalyzer(sample_metadata)
analyzer.summarize()
print(analyzer.summary)
# analyzer.plot_summary()
print(analyzer.summary.mesh_terms)

years={'2025': 20} journals={"Alzheimer's & dementia : the journal of the Alzheimer's Association": 7, 'Briefings in bioinformatics': 1, 'Journal of translational medicine': 1, 'Scientific reports': 2, 'Neuron': 1, 'Nature communications': 1, 'Translational psychiatry': 1, 'CNS neuroscience & therapeutics': 1, 'Bioinformatics (Oxford, England)': 1, 'Acta neuropathologica': 1, 'EMBO molecular medicine': 1, 'Analytical chemistry': 1, 'The journal of gene medicine': 1} mesh_terms={'Humans': 18, 'Alzheimer Disease': 20, 'DNA Methylation': 1, 'Aged': 14, 'Male': 16, 'Female': 15, 'Epigenomics': 1, 'Brain': 9, 'Epigenesis, Genetic': 1, 'Aged, 80 and over': 7, 'Genome, Human': 1, 'Retroelements': 1, 'Genomics': 5, 'Genetic Variation': 3, 'Animals': 4, 'Mice': 4, 'Disease Models, Animal': 4, 'Polymorphism, Single Nucleotide': 2, 'Cognition': 2, 'Binding Sites': 1, 'Quantitative Trait Loci': 1, 'Proteomics': 14, 'Mice, Transgenic': 4, 'Longitudinal Studies': 1, 'Cognitive Dysfunction': 4, 'Cere

## 7. Generate a Word Cloud from Article Titles

This step visualizes the most frequent and semantically meaningful phrases in article titles using the `LLMSummarizer` class. The `LLMSummarizer` leverages large language models (LLMs) for phrase extraction and, optionally, semantic clustering. When enabled, LLM-based embeddings group similar phrases together, allowing the word cloud to reflect not just raw frequency but also semantic similarity between phrases. This provides a more insightful view of the key topics and concepts present in the article titles.

In [7]:
# Generate a word cloud from article titles using LLMSummarizer
from niagads.nlp.analyzers import PhraseClusterAnalyzer

# Collect all article titles (use filtered_articles if available, else sample_metadata)
if 'filtered_articles' in locals() and filtered_articles:
    titles = [article.title for article in filtered_articles if article.title]
elif 'sample_metadata' in locals() and sample_metadata:
    titles = [article.title for article in sample_metadata if article.title]
else:
    titles = []

if titles:
    analyzer = PhraseClusterAnalyzer(top_n=30, min_ngram=1, max_ngram=3, use_embeddings=False, debug=DEBUG)
    clusters = analyzer.semantic_phrase_clustering(titles)
    analyzer.plot_ngram_wordcloud(clusters, max_words=50, title="Word Cloud of Article Titles")
else:
    print("No article titles available for word cloud visualization.")

## 8. Fetch Full Text for Selected PMIDs

Fetch full text XML for selected PMIDs from the PMC Open Access subset.

In [6]:
# Fetch full text for the first filtered article 
pmids = [f.pmid for f in filtered_articles]
# save to file
await pubmed_service.fetch_full_text(pmids, output_dir=os.getcwd())

# save to dict
full_text_xml = await pubmed_service.fetch_full_text(pmids)


2025-11-17 17:11:54,453 INFO niagads.common.core: PubMedQueryService:fetch_full_text      Fetching full text for 5 PMIDs (batch size = 200).
2025-11-17 17:11:54,457 DEBUG niagads.common.core: PubMedQueryService:_rate_limited_request Requesting: https://eutils.ncbi.nlm.nih.gov/entrez/eutils/elink.fcgi with params: {'dbfrom': 'pubmed', 'db': 'pmc', 'retmode': 'xml', 'id': '41178698,41174000,41085772,41078215,41073674'}
2025-11-17 17:11:54,749 DEBUG niagads.common.core: PubMedQueryService:_rate_limited_request Response received for /entrez/eutils/elink.fcgi: <?xml version="1.0" encoding="UTF-8" ?>
<!DOCTYPE eLinkResult PUBLIC "-//NLM//DTD elink 20101123//EN" "https://eutils.ncbi.nlm.nih.gov/eutils/dtd/20101123/elink.dtd">
<eLinkResult>

  <LinkSet>
    <DbFrom>pubmed</DbFrom>
    <IdList>
      <Id>41178698</Id>
      <Id>41174000</Id>
      <Id>41085772</Id>
      <Id>41078215</Id>
      <Id>41073674</Id>
    </IdList>
    <LinkSetDb>
      <DbTo>pmc</DbTo>
      <LinkName>pubmed_pmc</Li

## 9. Parse PMC Full Text XML

Create a PMCFullTextParser instance with the full text XML and parse it to extract content.

In [7]:
from niagads.pubmed.parsers import PMCFullTextParser

# Parse the full text XML if available
parser = PMCFullTextParser(full_text_xml=list(full_text_xml.values())[0])
parser.parse()
print("PMC full text parsed")


PMC full text parsed


## 10. Extract Tables, Sections, and Figures from Full Text

Use PMCFullTextParser methods to extract tables, sections, and figures from the parsed XML.

In [11]:
# Extract and display tables, sections, and figures from the parsed PMC full text
if parser:
    tables = parser.get_tables()
    sections = parser.get_sections()
    figures = parser.get_figures()
    print(f"Tables found: {len(tables)}")
    print(f"Sections found: {len(sections)}")
    print(f"Figures found: {len(figures)}")
    if tables:
        print("First table:", tables[0])
    if sections:
        print("First section:", sections[0])
        print(f"Section Titles: {[s['title'] for s in sections]}")
    if figures:
        print("First figure:", figures[0])
else:
    print("No parsed PMC full text to extract content from.")

Tables found: 0
Sections found: 8
Figures found: 7
First section: {'title': 'BACKGROUND', 'text': "The neuropathological changes of Alzheimer's disease (AD) begin two or more decades before signs of cognitive impairment, making early detection crucial for disease prevention and therapeutic engagement. Molecular subtyping has been proposed to address these issues, and recent efforts have utilized genomic, transcriptomic, proteomic, and metabolomic analyses to identify subtypes of AD associated with diverse phenotypes in the brain and biofluids. Here, we employed an unbiased network‐driven approach to characterize the CSF proteome, based not only on disease but also race, age, and sex. We analyzed 431 individuals across 483 samples, including 131 from African American participants, and organized the CSF proteome into 10 co‐expression modules linked to diverse molecular functions, pathways, and brain cell types. These modules were associated with age, sex, and race, as well as core AD bio

## 11. Extract Specific Sections: Results, Conclusions, Discussion

Extract and display only the sections with titles containing 'Results', 'Conclusions', or 'Discussion' from the parsed sections.

In [ ]:
# Extract and display 'Results', 'Conclusions', 'Discussion' sections using LLM-based synonym expansion
# if you run into issues w/the models, try deleting the cache:
# rm -rf ~/.cache/huggingface/hub
from niagads.nlp.core import NLPModel
from niagads.nlp.matchers.synonyms import LLMSynonymMatcher

if parser:
    # Define section titles to extract (case-insensitive)
    section_titles = ["Results", "Conclusions", "Discussion"]
    # Use LLMSynonymMatcher to get extended list of synonyms for section titles
    matcher = LLMSynonymMatcher(section_titles, model=NLPModel.T5)
    extended_synonyms =  await matcher.get_extended_synonyms("What are alternative section titles (no punctuation) in a research paper for:")
    print(f"Section titles + LLM synonyms: {extended_synonyms}")
    # Use parser.get_sections with the extended_synonyms as the titles argument
    filtered_sections = parser.get_sections(titles=list(extended_synonyms))
    print(f"Sections matching {section_titles} (with LLM synonyms): {len(filtered_sections)}")
    for sec in filtered_sections:
        print(f"Title: {sec['title']}")
        print(f"Text: {sec['text'][:500]}...\n")
else:
    print("No parsed PMC full text to extract specific sections from.")

2025-11-17 17:12:16,666 DEBUG urllib3.connectionpool: Starting new HTTPS connection (1): huggingface.co:443
2025-11-17 17:12:16,843 DEBUG urllib3.connectionpool: https://huggingface.co:443 "HEAD /Vamsi/T5_Paraphrase_Paws/resolve/main/tokenizer_config.json HTTP/1.1" 307 0
2025-11-17 17:12:16,865 DEBUG urllib3.connectionpool: https://huggingface.co:443 "HEAD /api/resolve-cache/models/Vamsi/T5_Paraphrase_Paws/f8c3dedd6b6f1bc7db90fee74c5338d6e0f99ba4/tokenizer_config.json HTTP/1.1" 200 0
2025-11-17 17:12:16,920 DEBUG urllib3.connectionpool: https://huggingface.co:443 "GET /api/models/Vamsi/T5_Paraphrase_Paws/tree/main/additional_chat_templates?recursive=False&expand=False HTTP/1.1" 404 64


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


2025-11-17 17:12:17,178 DEBUG urllib3.connectionpool: https://huggingface.co:443 "HEAD /Vamsi/T5_Paraphrase_Paws/resolve/main/config.json HTTP/1.1" 307 0
2025-11-17 17:12:17,194 DEBUG urllib3.connectionpool: https://huggingface.co:443 "HEAD /api/resolve-cache/models/Vamsi/T5_Paraphrase_Paws/f8c3dedd6b6f1bc7db90fee74c5338d6e0f99ba4/config.json HTTP/1.1" 200 0
2025-11-17 17:12:17,552 DEBUG urllib3.connectionpool: https://huggingface.co:443 "HEAD /Vamsi/T5_Paraphrase_Paws/resolve/main/generation_config.json HTTP/1.1" 404 0
2025-11-17 17:12:17,592 DEBUG urllib3.connectionpool: https://huggingface.co:443 "HEAD /Vamsi/T5_Paraphrase_Paws/resolve/main/custom_generate/generate.py HTTP/1.1" 404 0


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Section titles + LLM synonyms: {'results:', 'results :', 'results', 'discussion:', 'discussion :', 'conclusions :', 'conclusions', 'discussion', 'conclusions.'}
Sections matching ['Results', 'Conclusions', 'Discussion'] (with LLM synonyms): 2
Title: RESULTS
Text: We generated a consensus CSF proteomic dataset by harmonizing TMT‐MS proteomics across previously published studies. To investigate the impact of biological traits such as age, sex, and race on the broader AD CSF proteome, we employed an unbiased network‐based approach using WGCNA. This method clusters proteins based on shared expression patterns across individual samples, grouping them into biologically meaningful modules that reflect demographic traits (age, sex, and race), disease status, and...

Title: DISCUSSION
Text: This study highlights the significant molecular heterogeneity of AD by integrating a large, demographically diverse CSF proteomic dataset and identifying distinct biological subtypes. We show that establishe

## 12. Extract Gene References from Filtered Sections

Use the LLM-based gene extractor to identify gene mentions in the filtered sections. This step demonstrates how to apply the `GeneReferenceExtractor` to biomedical text extracted from PMC full text articles.

In [ ]:
# Extract gene references from the filtered sections using GeneReferenceExtractor
from niagads.nlp_utils.extractors.gene import GeneReferenceExtractor, GeneNERModel

gene_extractor = GeneReferenceExtractor(model=GeneNERModel.D4DATA)

# Combine all filtered section texts into one block for extraction
if parser and 'filtered_sections' in locals():
    all_section_text = "\n".join(sec['text'] for sec in filtered_sections)
    gene_mentions = gene_extractor.extract(all_section_text)
    print(f"Gene mentions found: {gene_mentions}")
else:
    print("No filtered sections available for gene extraction.")

## 13. Summarize Gene Contexts in Filtered Sections

Use the `TextSummarizer` to generate a summary for each gene, based on the sentences in which it appears in the filtered sections. This provides a concise overview of the context in which each gene is discussed.

In [ ]:
# Summarize gene contexts using TextSummarizer
from niagads.nlp_utils.summarizer import TextSummarizer, SummarizationModel

# Extract gene context sentences (per gene) from the filtered sections
if parser and 'filtered_sections' in locals():
    all_section_text = "\n".join(sec['text'] for sec in filtered_sections)
    gene_contexts = gene_extractor.extract_gene_contexts(all_section_text)
    summarizer = TextSummarizer(model=SummarizationModel.PEGASUS_PUBMED)
    gene_summaries = summarizer.summarize(gene_contexts)
    for gene, summary in gene_summaries.items():
        print(f"Gene: {gene}\nSummary: {summary}\n")
else:
    print("No filtered sections available for gene context summarization.")